
# Market Place Simulator
## I. Brief Explanation of the work flow and assumptions made

*Currency exchange rate varies over time:* 

    1. In order to simulate the exchange rate varying over time, I assume that the  exchange rate follows a Brownian motion type of curve (see ER_generator.m).  The exchange rate could automatically be obtained from the internet to get  the real time inter bank rate (IBR) from [link](http://www.fxstreet.com/rates-charts/forex-rates/).

    2. For every simulated time step, a random number of traders (following a  normal distribution) come to the system on both sides of the market. The  average of traders can be chosen, although the larger this number, the  longer it taks to run the simulation. The important fact is that the  average will change automatically following the inter-bank exchange rate.  In other words, if the IBR decreases for the trader in the GPB market,  there will be a smaller average number of people willing to sell Pounds,  whereas an increase in the average number of people in the Euro market  will be observed.

    3. The initial Euro to Pounds exchange rate is the latest given by the  european central bank ER(Euro/GBP= 0.77483). Likewise, the exchange rate  between Pounds and Euro is ER(GBP/Euro)=1/0.77483=1.2906.

    4. Quicktraders are those individuals that can not wait to have they  money exchanged, therefore they receive the best available rate.  Traders with quicktrader status are also included in the list of  traders and they are identified with a logical index.

    5. Each trader has an amount of cash. This is done by using a random  generator (following a uniform dist.). The mininum allowed is 100  (either GBP or Euros) and the maximum allowed is 500000 in multiples of 100.

*Building the Virtual Market:*

    6. For every time step, the lists with all the users on both markets are sorted with respect to their chosen ER. All the users with the same ER are then group and stored in separed cells with their sum of cash. The best_available ER on both sides of the market is determined by finding the group of users with approximately equal ER.

    7. The Matching_system function is called and the conditions for matching events are checked for those traders listed in the cells with the best_available rate. All the matching events are then saved into a new list and them into files.

    8. Priorities in matching events: 
    - traders with oldest TimeStampOrdered
    - when possible, quicktraders match with better rates with trades in the queue

*The market is updated after the conditions above are checked and a new
 best_available rate is calculated for the next simulation time.*


In [16]:
%format short
clear all
clc
disp('---------------------------------------------------------------')
disp(' WELCOME TO THE AMAZING CURRENCY-FAIR MARKET CURRENCY EXCHANGE!')
disp('===============================================================')

---------------------------------------------------------------
 WELCOME TO THE AMAZING CURRENCY-FAIR MARKET CURRENCY EXCHANGE!


## II. Initialization of global variables and parameters: 
We first need to defines the number of days that the generated data will correspond to.
Since we are running a *continuous time simulation*, we need to relate the real time to the unit of simulation time. This is achieved by  **real_time_ratio**. If you want to have  enough data for unmatched events within a particular number of days, you  should run a few extra days of simultion in order to generate those data.  For instance, if you want to train data for day_3, consider N_days>=5.

In [24]:
N_days=2;   %min_value=2
real_time_ratio=180;
time_length=(N_days*24*60*60)/real_time_ratio;  % This is the total time of simulation.
ER_market=1.2906;                % Real market exchange rate(RMER)
lowerb=1.2875;                   % Lower bound variation of RMER
upperb=1.2975;
delta=upperb-lowerb;             % Upper bound variation of RMER
avg_num_people_selling=3;        % Average number of new users selling GBP per unit 
avg_num_people_buying=3;         % Average number of new users buying GBP per unit time
min_amount_money=1;              % Min amount of money(multiples of 100, See "amount_per_user.m")
max_amount_money=1500;           % Max amount of money(multiples of 100, See "amount_per_user.m")
%
prob_QuickTr=[0.9 0.1];          % This vector defines the fraction of new users who decide
                                 % for a quicktrade. In this case, for every 10 new users,
                                 % one is a Quicktrader. (See "QuickTrade_gen.m")

## III. README: Source Code description
#### WARNING: depending on the number of days chosen, the calculation can take a few hours. In order to have a representative amount of data to predict matching events that takes 7 days, one should try to run *N_days* >=50.
All the files should be in the same directory. The entire source code can be found in the folder **Source_Code**. This these files in your folder and open **Matlab** to run the **P2P_EXCRATE.m**. This code will call all the other parts of the simulator.

    1. Parameters.m: is the file containing the parameters of the calculation, e.g., number of days to simulate, average number of traders, amount of cash each trader brings and so on. 

    2. ER_generator.m: creates a vector of size 1xtime_length for the exchange rate based on random number generator (following a brownian motion). The starting exchange rate is the real market exchange rate (ER_market). It represents the limits for variation of the brownian motion. 

    3. New_traders_list.m: This function contains many objects to create a new list of *traders* coming into the system on both sides of the market (sell and buying sides) and all the attributes quantities that defines a *trader*. It will calculate:
    - the number of traders 
    - the amount of money for each individual trader
    - the exchange rate for each individual trader
    - if the trader is *quicktrader* or not
    4. Matching_system.m: This object contains all the conditions for traders transactions to take place. The two markets (GBP-->Euro and Euro-->GBP) are evaluated simoutaneously. The *best_available* rate is the corresponding best available rate in the other market. 
    - The model will always maximize the users gain. If someone is willing to loose too much, he will have priority to get a better rate when is possible.
    - The users NEVER get rates below what they ordered. 
    - Queue_match.m: This function is called only by **Matching_system.m** function. If a matching event between two traders in local queues generated a partial match for one of them, then this function checks if it is possible to match the rest of cash of the trader in the queue with a quicktrader.
    - TradeLOG_generator.m: This function is called by **Matching_system.m** and it generates the data containing information on all the matching events that take place in the system and saves into the TradeLOG list wich will be then saved into a textfile. This is the headings of the data generated!
    5. Saving_data.m and Plot_data.m are called after the calculation has finished.

## IV. Data generated
After executing the **P2P_EXCRATE.m** code, many files will be generated that will be used in the second part (exploratory data analysis). Here is a quick summary of the files:
    1. dayX_allinfo.csv: contains all the logs for partial matching, i.e., if a trader has secured a partial matching with one trader on the other side and the rest of his money with a second trader. X corresponds to the respective day, in this case we are saving files just up to matching events taking 7 days to occur.
    2. dayX_relinfo.csv: the same as before, however all the partial matching events for a particular user is aggregated as one single entry. Therefore, the ER is the averaged ER for all his partial matchings. Also, the amount of money is the today originally ordered.
    3. LinearReg_relinfo.csv: contains the same information as 2. however without the logical variable *Matched_day*.
The files mentioned above are already balanced data, i.e., they are saved where the amount of data for no-matching is the same as for matching (50/50). 
    4. TradeLog_all.csv: is the main data set. It contains all the individual partial matching events for all users with a unique TradeID. 